<a href="https://colab.research.google.com/github/PTC-Education/PTC-API-Playground/blob/main/Google_Sheets_AL5D_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onshape-client
from onshape_client.client import Client
import json
base = 'https://cad.onshape.com' # change this if you're using a document in an enterprise (i.e. "https://ptc.onshape.com")

     |████████████████████████████████| 2.1MB 24.4MB/s 
     |████████████████████████████████| 112kB 53.9MB/s 
     |████████████████████████████████| 215kB 46.9MB/s 
     |████████████████████████████████| 552kB 38.3MB/s 


In [ ]:
import os
def include(filename):
    if os.path.exists(filename): 
        execfile(filename)
include('colabkeys.py')
client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})
print('client configured')

client configured


## Digital Twin Functions

In [ ]:
def setMateValues(baseAngle,shoulderAngle,elbowAngle,wristAngle,handAngle,gripperAngle):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'

  # https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/2f52bf4870f9d7ddc900b4de
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'
  eid = '2f52bf4870f9d7ddc900b4de'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  fullResponse = json.loads(response.data)
  
  for i in range(len(fullResponse["mateValues"])):
    if fullResponse['mateValues'][i]['mateName'] == "Base":
      print(fullResponse['mateValues'][i])
      fullResponse['mateValues'][i]['rotationZ'] = baseAngle
      print(fullResponse['mateValues'][i])

  method = 'POST'

  params = {}
  payload = fullResponse
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}
  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  for i in range(len(fullResponse["mateValues"])):
    if fullResponse['mateValues'][i]['mateName'] == "Shoulder":
      fullResponse['mateValues'][i]['rotationZ'] = shoulderAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Elbow":
      fullResponse['mateValues'][i]['rotationZ'] = elbowAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Wrist":
      fullResponse['mateValues'][i]['rotationZ'] = wristAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Hand":
      fullResponse['mateValues'][i]['rotationZ'] = handAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Gripper":
      fullResponse['mateValues'][i]['rotationZ'] = gripperAngle

  method = 'POST'

  params = {}
  payload = fullResponse
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}
  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  # The command below prints the entire JSON response from Onshape
  print(response.status)

In [ ]:
def getTCPPos():
  import array
  base = 'https://cad.onshape.com'
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid'

  # https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/2f52bf4870f9d7ddc900b4de
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'
  eid = '2f52bf4870f9d7ddc900b4de'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))

  for i in range(len(parsed['rootAssembly']['instances'])):
    if "TCP" in parsed['rootAssembly']['instances'][i]['name']:
      TCPid = parsed['rootAssembly']['instances'][i]['id']
      break

  for i in range(len(parsed['rootAssembly']['occurrences'])):
    if parsed['rootAssembly']['occurrences'][i]['path'][0] == TCPid and len(parsed['rootAssembly']['occurrences'][i]['path'])==1:
      TCPpos = [parsed['rootAssembly']['occurrences'][i]['transform'][3],parsed['rootAssembly']['occurrences'][i]['transform'][7],parsed['rootAssembly']['occurrences'][i]['transform'][11]]
  return TCPpos
print(getTCPPos())

[0.2954604357280716, 0.011017799713790981, 0.20607492077575645]


## Connect Google Sheets

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1-NF7KqZnMjmmixTiP35Eibw5LOw87BVjygtL9KePYVU').sheet1

In [ ]:
def SetCellsFromTCP():
  TCP_cell_list = worksheet.range('B2:D2')
  TCP_pos = getTCPPos()
  for i in range(len(TCP_cell_list)):
    TCP_cell_list[i].value = TCP_pos[i]
  worksheet.update_cells(TCP_cell_list)

In [ ]:
def SetMatesFromCells():
  Joint_cell_list = worksheet.range('B5:G5')
  setMateValues(Joint_cell_list[0].value,Joint_cell_list[1].value,Joint_cell_list[2].value,Joint_cell_list[3].value,Joint_cell_list[4].value,Joint_cell_list[5].value)

In [ ]:
import math
setMateValues(math.pi/2,math.pi/2,math.pi/2,math.pi/2,math.pi/2,math.pi/3)

{'jsonType': 'Revolute', 'rotationZ': 1.5707963267948835, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': 1.5707963267948966, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200


In [ ]:
import time

while True:
  SetCellsFromTCP()
  time.sleep(0.5)
  SetMatesFromCells()
  time.sleep(0.5)

{'jsonType': 'Revolute', 'rotationZ': 1.5706999999999893, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': '1.5707', 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200
{'jsonType': 'Revolute', 'rotationZ': 1.5707000000000435, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': '1.5707', 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200
{'jsonType': 'Revolute', 'rotationZ': 1.5707000000000249, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': '1.3962', 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200
